## 네이버 카페 글 제목, 내용 크롤링

In [ ]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service as ChromeService
from webdriver_manager.chrome import ChromeDriverManager
import pandas as pd
import time

# Chrome WebDriver 설정
options = webdriver.ChromeOptions()
driver = webdriver.Chrome(service=ChromeService(ChromeDriverManager().install()), options=options)

# URL 설정
url = 'https://cafe.naver.com/livehope?iframe_url=/ArticleList.nhn%3Fsearch.clubid=14422182%26search.menuid=85%26search.boardtype=L'
driver.get(url)

# 페이지 로드 대기
time.sleep(2)  # 페이지 로딩 시간을 고려하여 적절히 조정

# 스크롤을 내려서 iframe을 로드해야 할 경우가 있으므로 스크롤 추가
driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
time.sleep(1)

# iframe으로 전환
driver.switch_to.frame("cafe_main")

# 결과 저장 리스트
titles_and_contents = []

# 10페이지 반복
for page in range(5):
    for i in range(1, 16):
        try:
            # 글 제목 선택자
            selector = f'#main-area > div:nth-child(4) > table > tbody > tr:nth-child({i}) > td.td_article > div.board-list > div > a.article'
            title_element = driver.find_element(By.CSS_SELECTOR, selector)
            title = title_element.text
            title_element.click()
            
            # 글 내용 선택자
            time.sleep(60)  # 페이지 로딩 대기
            content_element = driver.find_element(By.CSS_SELECTOR, '#app > div > div > div.ArticleContentBox > div.article_container > div.article_viewer')
            content = content_element.text

            # 페이지의 3/4 지점까지 스크롤
            driver.execute_script("window.scrollTo(0, document.body.scrollHeight * 0.75);")

            
            # 결과 저장
            titles_and_contents.append({
                "글 제목": title,
                "글 내용": content
            })
            
            # 뒤로 가기
            driver.back()
            time.sleep(2)
            
            # 다시 iframe으로 전환
            driver.switch_to.frame("cafe_main")
        except Exception as e:
            print(f"Error processing element {i} on page {page + 1}: {e}")
    
    try:
        # 다음 페이지로 이동
        next_button = driver.find_element(By.CSS_SELECTOR, '#main-area > div.prev-next > a.pgR')
        next_button.click()
        time.sleep(2)  # 페이지 로딩 대기
    except Exception as e:
        print(f"Error moving to next page {page + 1}: {e}")
        break

# DataFrame으로 변환
df = pd.DataFrame(titles_and_contents)

# DataFrame 출력
print(df)

# DataFrame 저장
# output_file_path = 'naver_cafe_articles.xlsx'
# df.to_excel(output_file_path, index=False)
# print(f"DataFrame이 {output_file_path}에 저장되었습니다.")


In [5]:
df

,글 제목,글 내용
0,북미스탈 왕파김치 담기,"1. 게시글, 댓글, 닉네임에 초성을 사용하지 마세요.\n2. 본인 게시글을 삭제하..."
1,배추겉절이 했어요~~,"1. 게시글, 댓글, 닉네임에 초성을 사용하지 마세요.\n2. 본인 게시글을 삭제하..."
2,뚝딱요리 순두부명란계란탕 뜨끈한국물이 생각날때!!,"1. 게시글, 댓글, 닉네임에 초성을 사용하지 마세요.\n2. 본인 게시글을 삭제하..."
3,후무스&리코타치즈만들어 샐러드에 올려보자,"1. 게시글, 댓글, 닉네임에 초성을 사용하지 마세요.\n2. 본인 게시글을 삭제하..."
4,NO 밀가루 돌돌배추전,"1. 게시글, 댓글, 닉네임에 초성을 사용하지 마세요.\n2. 본인 게시글을 삭제하..."
5,함박눈이 내리는 날은 손칼국수~~^^,"1. 게시글, 댓글, 닉네임에 초성을 사용하지 마세요.\n2. 본인 게시글을 삭제하..."
6,드디어 곰탕 완료~~^^,"1. 게시글, 댓글, 닉네임에 초성을 사용하지 마세요.\n2. 본인 게시글을 삭제하..."
7,추운 날씨에 딱 어울리는 음식,"1. 게시글, 댓글, 닉네임에 초성을 사용하지 마세요.\n2. 본인 게시글을 삭제하..."
8,오늘은 친정 김장 하는 날~~,"1. 게시글, 댓글, 닉네임에 초성을 사용하지 마세요.\n2. 본인 게시글을 삭제하..."


In [3]:
# DataFrame 저장
output_file_path = r'C:\Users\USER\Desktop\5_6\core_pj\data\naver_cafe_articles.xlsx'
df.to_excel(output_file_path, index=False)
print(f"DataFrame이 {output_file_path}에 저장되었습니다.")

DataFrame이 C:\Users\USER\Desktop\5_6\core_pj\data\naver_cafe_articles.xlsx에 저장되었습니다.


# !!! 비상 상황 !!!

원래 위 코드처럼 하면 크롤링에 문제가 없지만, 네이버 측에서 딱 10번째 글을 조회하는 순간 로그인 창이 뜬다. <br>
time sleep을 30초로 설정해도 떠. 그렇다고 60초로 늘리기에는 차라리 직접 다 복사 붙여넣기가 빠르지.

8개, 7개씩 긁고 df에 더해보는 식으로 가보자.
8개 긁고
#main-area > div:nth-child(4) > table > tbody > tr:nth-child(1) > td.td_article > div.board-list > div > a.article
여기서부터 
서 
#main-area > div:nth-child(4) > table > tbody > tr:nth-child(8) > td.td_article > div.board-list > div > a.article
여기까지 8개 긁어서 df에 더할거야.

그리고 새코드를 실행해서
7개를 긁고 다음 버튼을 눌러서 넘어가고 


In [8]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service as ChromeService
from webdriver_manager.chrome import ChromeDriverManager
import pandas as pd
import time

def initialize_driver():
    options = webdriver.ChromeOptions()
    driver = webdriver.Chrome(service=ChromeService(ChromeDriverManager().install()), options=options)
    return driver

def crawl_articles(start, end, page):
    global driver
    articles = []
    for i in range(start, end + 1):
        try:
            # 글 제목 선택자
            selector = f'#main-area > div:nth-child(4) > table > tbody > tr:nth-child({i}) > td.td_article > div.board-list > div > a.article'
            title_element = driver.find_element(By.CSS_SELECTOR, selector)
            title = title_element.text
            title_element.click()
            
            # 글 내용 선택자
            time.sleep(2)  # 페이지 로딩 대기
            content_element = driver.find_element(By.CSS_SELECTOR, '#app > div > div > div.ArticleContentBox > div.article_container > div.article_viewer')
            content = content_element.text
            
            # 결과 저장
            articles.append({
                "글 제목": title,
                "글 내용": content
            })
            
            # 뒤로 가기
            driver.back()
            time.sleep(2)
            
            # 다시 iframe으로 전환
            driver.switch_to.frame("cafe_main")
        except Exception as e:
            print(f"Error processing element {i} on page {page + 1}: {e}")
    return articles

# 초기화
titles_and_contents = []
url = 'https://cafe.naver.com/livehope?iframe_url=/ArticleList.nhn%3Fsearch.clubid=14422182%26search.menuid=85%26search.boardtype=L'
driver = initialize_driver()

# 5페이지 반복
for page in range(5):
    # URL 설정 및 페이지 로드
    driver.get(url)
    time.sleep(2)
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    time.sleep(1)
    driver.switch_to.frame("cafe_main")

    # 첫 8개 글 크롤링
    titles_and_contents.extend(crawl_articles(1, 8, page))

    # 드라이버를 재설정하여 다음 7개 글 크롤링
    driver.quit()
    driver = initialize_driver()
    driver.get(url)
    time.sleep(2)
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    time.sleep(1)
    driver.switch_to.frame("cafe_main")
    titles_and_contents.extend(crawl_articles(9, 15, page))

    try:
        # 다음 페이지 URL 가져오기
        next_button = driver.find_element(By.CSS_SELECTOR, '#main-area > div.prev-next > a.pgR')
        next_url = next_button.get_attribute('href')
        url = next_url
    except Exception as e:
        print(f"Error moving to next page {page + 1}: {e}")
        break

# DataFrame으로 변환
df = pd.DataFrame(titles_and_contents)

# DataFrame 출력
print(df)

# DataFrame 저장 (필요에 따라 주석 해제)
output_file_path = 'naver_cafe_articles.xlsx'
df.to_excel(output_file_path, index=False)
print(f"DataFrame이 {output_file_path}에 저장되었습니다.")


Error processing element 10 on page 1: Message: no such element: Unable to locate element: {"method":"css selector","selector":"#app > div > div > div.ArticleContentBox > div.article_container > div.article_viewer"}
  (Session info: chrome=124.0.6367.210); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#no-such-element-exception
Stacktrace:
	GetHandleVerifier [0x0118C113+48259]
	(No symbol) [0x0111CA41]
	(No symbol) [0x01010A17]
	(No symbol) [0x01050BED]
	(No symbol) [0x01050C9B]
	(No symbol) [0x0108BC12]
	(No symbol) [0x01070DE4]
	(No symbol) [0x01089B9C]
	(No symbol) [0x01070B36]
	(No symbol) [0x0104570D]
	(No symbol) [0x010462CD]
	GetHandleVerifier [0x014465A3+2908435]
	GetHandleVerifier [0x01483BBB+3159851]
	GetHandleVerifier [0x012250CB+674875]
	GetHandleVerifier [0x0122B28C+699900]
	(No symbol) [0x01126244]
	(No symbol) [0x01122298]
	(No symbol) [0x0112242C]
	(No symbol) [0x01114BB0]
	BaseThreadInitThunk [0x76

## 이것도 안 되니, 다른 방식으로 해보자

In [14]:
import pandas as pd

# URL 정의
url = 'https://cafe.naver.com/livehope?iframe_url=/ArticleList.nhn%3Fsearch.clubid=14422182%26search.menuid=85%26search.boardtype=L'

# 빈 DataFrame 초기화
df = pd.DataFrame(columns=["글 제목", "글 내용"])

In [ ]:
url
df

In [15]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service as ChromeService
from webdriver_manager.chrome import ChromeDriverManager
import pandas as pd
import time

# Chrome WebDriver 설정
options = webdriver.ChromeOptions()
driver = webdriver.Chrome(service=ChromeService(ChromeDriverManager().install()), options=options)

# URL 설정 및 페이지 로드
driver.get(url)
time.sleep(2)  # 페이지 로드 대기

# 스크롤을 내려서 iframe을 로드해야 할 경우가 있으므로 스크롤 추가
driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
time.sleep(1)

# iframe으로 전환
driver.switch_to.frame("cafe_main")

# 8개의 글 크롤링
titles_and_contents = []
for i in range(1, 9):
    try:
        # 글 제목 선택자
        selector = f'#main-area > div:nth-child(4) > table > tbody > tr:nth-child({i}) > td.td_article > div.board-list > div > a.article'
        title_element = driver.find_element(By.CSS_SELECTOR, selector)
        title = title_element.text
        title_element.click()

        # 글 내용 선택자
        time.sleep(2)  # 페이지 로딩 대기
        content_element = driver.find_element(By.CSS_SELECTOR, '#app > div > div > div.ArticleContentBox > div.article_container > div.article_viewer')
        content = content_element.text

        # 결과 저장
        titles_and_contents.append({
            "글 제목": title,
            "글 내용": content
        })

        # 뒤로 가기
        driver.back()
        time.sleep(2)

        # 다시 iframe으로 전환
        driver.switch_to.frame("cafe_main")
    except Exception as e:
        print(f"Error processing element {i}: {e}")

# DataFrame에 추가
new_df = pd.DataFrame(titles_and_contents)
df = pd.concat([df, new_df], ignore_index=True)

# 지금까지의 DataFrame 출력
print(df)

# 드라이버 종료
driver.quit()


                          글 제목  \
0                 북미스탈 왕파김치 담기   
1                  배추겉절이 했어요~~   
2  뚝딱요리 순두부명란계란탕 뜨끈한국물이 생각날때!!   
3       후무스&리코타치즈만들어 샐러드에 올려보자   
4                 NO 밀가루 돌돌배추전   
5         함박눈이 내리는 날은 손칼국수~~^^   
6                드디어 곰탕 완료~~^^   
7             추운 날씨에 딱 어울리는 음식   

                                                글 내용  
0  1. 게시글, 댓글, 닉네임에 초성을 사용하지 마세요.\n2. 본인 게시글을 삭제하...  
1  1. 게시글, 댓글, 닉네임에 초성을 사용하지 마세요.\n2. 본인 게시글을 삭제하...  
2  1. 게시글, 댓글, 닉네임에 초성을 사용하지 마세요.\n2. 본인 게시글을 삭제하...  
3  1. 게시글, 댓글, 닉네임에 초성을 사용하지 마세요.\n2. 본인 게시글을 삭제하...  
4  1. 게시글, 댓글, 닉네임에 초성을 사용하지 마세요.\n2. 본인 게시글을 삭제하...  
5  1. 게시글, 댓글, 닉네임에 초성을 사용하지 마세요.\n2. 본인 게시글을 삭제하...  
6  1. 게시글, 댓글, 닉네임에 초성을 사용하지 마세요.\n2. 본인 게시글을 삭제하...  
7  1. 게시글, 댓글, 닉네임에 초성을 사용하지 마세요.\n2. 본인 게시글을 삭제하...  


In [16]:
# Chrome WebDriver 재설정
driver = webdriver.Chrome(service=ChromeService(ChromeDriverManager().install()), options=options)

# URL 설정 및 페이지 로드
driver.get(url)
time.sleep(2)  # 페이지 로드 대기

# 스크롤을 내려서 iframe을 로드해야 할 경우가 있으므로 스크롤 추가
driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
time.sleep(1)

# iframe으로 전환
driver.switch_to.frame("cafe_main")

# 7개의 글 크롤링
titles_and_contents = []
for i in range(9, 16):
    try:
        # 글 제목 선택자
        selector = f'#main-area > div:nth-child(4) > table > tbody > tr:nth-child({i}) > td.td_article > div.board-list > div > a.article'
        title_element = driver.find_element(By.CSS_SELECTOR, selector)
        title = title_element.text
        title_element.click()

        # 글 내용 선택자
        time.sleep(2)  # 페이지 로딩 대기
        content_element = driver.find_element(By.CSS_SELECTOR, '#app > div > div > div.ArticleContentBox > div.article_container > div.article_viewer')
        content = content_element.text

        # 결과 저장
        titles_and_contents.append({
            "글 제목": title,
            "글 내용": content
        })

        # 뒤로 가기
        driver.back()
        time.sleep(2)

        # 다시 iframe으로 전환
        driver.switch_to.frame("cafe_main")
    except Exception as e:
        print(f"Error processing element {i}: {e}")

# DataFrame에 추가
new_df = pd.DataFrame(titles_and_contents)
df = pd.concat([df, new_df], ignore_index=True)

# 지금까지의 DataFrame 출력
print(df)

# 다음 페이지로 이동
try:
    next_button = driver.find_element(By.CSS_SELECTOR, '#main-area > div.prev-next > a.pgR')
    next_url = next_button.get_attribute('href')
    url = next_url
    print(f"Next page URL: {url}")
except Exception as e:
    print(f"Error moving to next page: {e}")

# 드라이버 종료 (필요에 따라 주석 처리)
# driver.quit()


Error processing element 10: Message: no such element: Unable to locate element: {"method":"css selector","selector":"#app > div > div > div.ArticleContentBox > div.article_container > div.article_viewer"}
  (Session info: chrome=124.0.6367.210); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#no-such-element-exception
Stacktrace:
	GetHandleVerifier [0x0118C113+48259]
	(No symbol) [0x0111CA41]
	(No symbol) [0x01010A17]
	(No symbol) [0x01050BED]
	(No symbol) [0x01050C9B]
	(No symbol) [0x0108BC12]
	(No symbol) [0x01070DE4]
	(No symbol) [0x01089B9C]
	(No symbol) [0x01070B36]
	(No symbol) [0x0104570D]
	(No symbol) [0x010462CD]
	GetHandleVerifier [0x014465A3+2908435]
	GetHandleVerifier [0x01483BBB+3159851]
	GetHandleVerifier [0x012250CB+674875]
	GetHandleVerifier [0x0122B28C+699900]
	(No symbol) [0x01126244]
	(No symbol) [0x01122298]
	(No symbol) [0x0112242C]
	(No symbol) [0x01114BB0]
	BaseThreadInitThunk [0x763C7BA9+25]

## 자자 결국 원점이다. 10번째 조회 시 네이버 로그인이 뜨는 현상을 방지하고자...!!!

In [17]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service as ChromeService
from webdriver_manager.chrome import ChromeDriverManager
import pandas as pd
import time

# 네이버 로그인 정보 설정
naver_id = 'testwlsdn'
naver_pw = 'xptmxmT1q!'

# Chrome WebDriver 설정
options = webdriver.ChromeOptions()
driver = webdriver.Chrome(service=ChromeService(ChromeDriverManager().install()), options=options)

# 네이버 로그인
driver.get('https://nid.naver.com/nidlogin.login')
time.sleep(2)  # 페이지 로드 대기

driver.find_element(By.ID, 'id').send_keys(naver_id)
driver.find_element(By.ID, 'pw').send_keys(naver_pw)
driver.find_element(By.ID, 'log.login').click()
time.sleep(2)  # 로그인 처리 대기

# URL 설정 및 페이지 로드
url = 'https://cafe.naver.com/livehope?iframe_url=/ArticleList.nhn%3Fsearch.clubid=14422182%26search.menuid=85%26search.boardtype=L'
driver.get(url)
time.sleep(2)  # 페이지 로드 대기

# 스크롤을 내려서 iframe을 로드해야 할 경우가 있으므로 스크롤 추가
driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
time.sleep(1)

# iframe으로 전환
driver.switch_to.frame("cafe_main")

# 결과 저장 리스트
titles_and_contents = []

# 10페이지 반복
for page in range(5):
    for i in range(1, 16):
        try:
            # 글 제목 선택자
            selector = f'#main-area > div:nth-child(4) > table > tbody > tr:nth-child({i}) > td.td_article > div.board-list > div > a.article'
            title_element = driver.find_element(By.CSS_SELECTOR, selector)
            title = title_element.text
            title_element.click()
            
            # 글 내용 선택자
            time.sleep(2)  # 페이지 로딩 대기
            content_element = driver.find_element(By.CSS_SELECTOR, '#app > div > div > div.ArticleContentBox > div.article_container > div.article_viewer')
            content = content_element.text

            # 페이지의 3/4 지점까지 스크롤
            driver.execute_script("window.scrollTo(0, document.body.scrollHeight * 0.75);")

            # 결과 저장
            titles_and_contents.append({
                "글 제목": title,
                "글 내용": content
            })
            
            # 뒤로 가기
            driver.back()
            time.sleep(2)
            
            # 다시 iframe으로 전환
            driver.switch_to.frame("cafe_main")
        except Exception as e:
            print(f"Error processing element {i} on page {page + 1}: {e}")
    
    try:
        # 다음 페이지로 이동
        next_button = driver.find_element(By.CSS_SELECTOR, '#main-area > div.prev-next > a.pgR')
        next_button.click()
        time.sleep(2)  # 페이지 로딩 대기
    except Exception as e:
        print(f"Error moving to next page {page + 1}: {e}")
        break

# DataFrame으로 변환
df = pd.DataFrame(titles_and_contents)

# DataFrame 출력
print(df)

# DataFrame 저장
output_file_path = 'naver_cafe_articles.xlsx'
df.to_excel(output_file_path, index=False)
print(f"DataFrame이 {output_file_path}에 저장되었습니다.")

# 드라이버 종료
# driver.quit()


Error processing element 10 on page 1: Message: no such element: Unable to locate element: {"method":"css selector","selector":"#app > div > div > div.ArticleContentBox > div.article_container > div.article_viewer"}
  (Session info: chrome=124.0.6367.210); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#no-such-element-exception
Stacktrace:
	GetHandleVerifier [0x0118C113+48259]
	(No symbol) [0x0111CA41]
	(No symbol) [0x01010A17]
	(No symbol) [0x01050BED]
	(No symbol) [0x01050C9B]
	(No symbol) [0x0108BC12]
	(No symbol) [0x01070DE4]
	(No symbol) [0x01089B9C]
	(No symbol) [0x01070B36]
	(No symbol) [0x0104570D]
	(No symbol) [0x010462CD]
	GetHandleVerifier [0x014465A3+2908435]
	GetHandleVerifier [0x01483BBB+3159851]
	GetHandleVerifier [0x012250CB+674875]
	GetHandleVerifier [0x0122B28C+699900]
	(No symbol) [0x01126244]
	(No symbol) [0x01122298]
	(No symbol) [0x0112242C]
	(No symbol) [0x01114BB0]
	BaseThreadInitThunk [0x76

In [18]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service as ChromeService
from webdriver_manager.chrome import ChromeDriverManager
import time

# 네이버 로그인 정보 설정
naver_id = 'YOUR_NAVER_ID'
naver_pw = 'YOUR_NAVER_PASSWORD'

# Chrome WebDriver 설정
options = webdriver.ChromeOptions()
driver = webdriver.Chrome(service=ChromeService(ChromeDriverManager().install()), options=options)

# 네이버 로그인 페이지 접근
driver.get('https://nid.naver.com/nidlogin.login')
time.sleep(2)  # 페이지 로드 대기

# 아이디와 비밀번호 입력
driver.find_element(By.ID, 'id').send_keys(naver_id)
driver.find_element(By.ID, 'pw').send_keys(naver_pw)

# 로그인 버튼 클릭
driver.find_element(By.ID, 'log.login').click()
time.sleep(2)  # 로그인 처리 대기

print("로그인 완료")


로그인 완료
